## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 475289dcc8b84aca8ab0b93240548fa7, rmse: 6.3040
run id: 4594ef3cb9484b0b9681b071dd5cf359, rmse: 6.3097
run id: 474cee35444540e38e65598c6c4add29, rmse: 6.3668
run id: d16e8ab4730942baaab1b3efc9ec4ade, rmse: 6.3727
run id: 82a3d9d98340403fb0fa7ac7fdb19e21, rmse: 6.4190


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
#run_id = "b8904012c84343b5bf8ee72aa8f0f402"
run_id = "475289dcc8b84aca8ab0b93240548fa7"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/28 16:26:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653747987706, current_stage='None', description=None, last_updated_timestamp=1653747987706, name='nyc-taxi-regressor', run_id='475289dcc8b84aca8ab0b93240548fa7', run_link=None, source='./mlruns/1/475289dcc8b84aca8ab0b93240548fa7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [10]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653746905276, description='nyc taxi predictor for trip duration', last_updated_timestamp=1653747987706, latest_versions=[<ModelVersion: creation_timestamp=1653746905301, current_stage='Staging', description='', last_updated_timestamp=1653747143916, name='nyc-taxi-regressor', run_id='475289dcc8b84aca8ab0b93240548fa7', run_link='', source='./mlruns/1/475289dcc8b84aca8ab0b93240548fa7/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>,
  <ModelVersion: creation_timestamp=1653747987706, current_stage='None', description=None, last_updated_timestamp=1653747987706, name='nyc-taxi-regressor', run_id='475289dcc8b84aca8ab0b93240548fa7', run_link=None, source='./mlruns/1/475289dcc8b84aca8ab0b93240548fa7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='nyc-taxi-regressor', tags={}>]

In [11]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [12]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653747987706, current_stage='Staging', description=None, last_updated_timestamp=1653748414965, name='nyc-taxi-regressor', run_id='475289dcc8b84aca8ab0b93240548fa7', run_link=None, source='./mlruns/1/475289dcc8b84aca8ab0b93240548fa7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [13]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653747987706, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2022-05-28', last_updated_timestamp=1653748506608, name='nyc-taxi-regressor', run_id='475289dcc8b84aca8ab0b93240548fa7', run_link=None, source='./mlruns/1/475289dcc8b84aca8ab0b93240548fa7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [29]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    #model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    #print(model)
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [30]:
df = read_dataframe("../01-intro/data/green_tripdata_2021-03.parquet")

In [31]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'c:\\Users\\hansk\\kurser\\mlops_zoomcamp\\02-experiment-tracking\\preprocessor'

In [32]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:
X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [36]:
model_name

'nyc-taxi-regressor'

In [40]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[17:20:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 14.7 s
Wall time: 692 ms


{'rmse': 6.250963441118867}

In [39]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[17:19:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 13.5 s
Wall time: 825 ms


{'rmse': 6.250963441118867}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>